<div dir='rtl'>
<h1>پروژه اول</h1>
</div>

In [34]:
from timeit import default_timer as timer
from copy import deepcopy, copy
from typing import Callable, Union
from dataclasses import dataclass
from collections import deque
import heapq

In [35]:
INPUT_COUNT = 5
INPUT_PATH  = 'data/input%d.txt'
TIMER_TEST_COUNT = 3

In [36]:
@dataclass
class Graph:
    n: int
    m: int
    current: int
    cost: int
    parent: Union['Graph', None]
    adj: list[set[int]]
    difficulties: dict[int, int]
    requirements: dict[int, set[int]]
    remainingTime: int
    
    def __init__(self, n: int, m: int):
        self.n = n
        self.m = m
        self.current = 0
        self.cost = 0
        self.parent = None
        self.adj = [set() for _ in range(n)]
        self.difficulties = {}
        self.requirements = {}
        self.remainingTime = 0

    def __deepcopy__(self, memo: dict) -> 'Graph':
        g = Graph(self.n, self.m)
        g.current = self.current
        g.cost = self.cost
        g.parent = self.parent
        g.adj = copy(self.adj)
        g.difficulties = deepcopy(self.difficulties)
        g.requirements = deepcopy(self.requirements)
        g.remainingTime = self.remainingTime
        return g

    def __lt__(self, other: 'Graph') -> bool:
        return self.cost < other.cost

    def __key(self) -> tuple[int, dict[int, int], dict[int, set[int]]]:
        return (self.current, self.difficulties, self.requirements)

    def __str__(self) -> str:
        return str(self.__key())

    def __hash__(self) -> int:
        return hash(self.__str__())

<div dir='rtl'>
برای مدل کردن مسئله، هر 
state
را با یک گراف نمایش می‌دهیم. 
این
stateها 
شامل مرید‌های باقی‌مانده و دستور غذاهای باقی مانده آن‌ها، مکان فعلی سید در نقشه، وضعیت گذر از رئوس صعب العبور و زمان باقی مانده برای خروج از راس فعلی (در صورتی که راس فعلی صعب العبور باشد) است.
همچنین در هر 
state
مقادیر زیر را نیز نگهداری می‌کنیم:
<ul>
    <li>هزینه رسیدن به state فعلی</li>
    <li>پدر state فعلی</li>
    <li>یال‌های موجود در گراف</li>
</ul>
دو
state
را در صورتی یکسان در نظر می‌گیریم که موارد زیر در آن‌ها یکسان باشند:
<ol>
    <li>مکان فعلی سید</li>
    <li>مریدهای باقی‌مانده و دستور غذاهای باقی‌مانده برای هر مرید</li>
    <li>وضعیت گذر از رئوس صعب العبور</li>
</ol>
تست
goal
را به این صورت در نظر می‌گیریم که هیچ مرید و دستور غذایی باقی نمانده باشد.<br/>
همچنین برای 
actionها،
از هر راس می‌توانیم به راس همسایه برویم و یا اینکه یک واحد از زمان باقی‌مانده حضور در مکان فعلی (راس صعب العبور) کم کنیم.
زمانی که به راس همسایه می‌رویم، بررسی می‌کنیم که در آن دستور غذا حفظ نشده و یا مریدی که تمام دستور غذاهای آن حفظ شده وجود دارد یا خیر، در صورت وجود، تغییرات لازم را در 
state
جدید اعمال می‌کنیم.<br/>
همچنین
initial state
را گراف ورودی سوال با تمام ویژگی‌های ورودی در نظر می‌گیریم.
</div>

In [37]:
def readInput(path: str) -> Graph:
    with open(path, 'r', encoding='utf-8') as f:
        n, m = map(int, f.readline().split())
        graph = Graph(n, m)
        for _ in range(m):
            u, v = map(int, f.readline().split())
            graph.adj[u - 1].add(v - 1)
            graph.adj[v - 1].add(u - 1)
        _ = int(f.readline())
        difficulties = map(int, f.readline().split())
        for d in difficulties:
            graph.difficulties[d - 1] = -1
        s = int(f.readline())
        for _ in range(s):
            inp = f.readline().split()
            p = int(inp[0]) - 1
            vertices = {int(x) - 1 for x in inp[2:]}
            graph.requirements[p] = vertices
        i = int(f.readline()) - 1
        graph.current = i
        if i in graph.difficulties:
            graph.difficulties[i] += 1
        for u, vertices in graph.requirements.items():
            if i in vertices:
                vertices.remove(i)
        if i in graph.requirements and len(graph.requirements[i]) == 0:
            del graph.requirements[i]
    return graph

In [38]:
inputs = [readInput(INPUT_PATH % i) for i in range(1, INPUT_COUNT + 1)]

<div dir='rtl'>
ورودی‌ها در فولدر 
data
و به شکل
input(i).txt
قرار گرفته‌اند. شماره تست‌ها
(i)
به صورت زیر است:
<ol>
    <li>تست شماره 1 در تست کیس‌های اصلی و ساده</li>
    <li>تست شماره 2 در تست کیس‌های ساده</li>
    <li>تست شماره 3 در تست کیس‌های ساده</li>
    <li>تست شماره 2 در تست کیس‌های اصلی</li>
    <li>تست شماره 3 در تست کیس‌های اصلی</li>
</ol>
</div>

In [39]:
def searchTime(g: Graph, method: Callable[[Graph], tuple[list[int] | None, int | None, int]]) -> float:
    sum = 0
    for _ in range(TIMER_TEST_COUNT):
        gCopy = deepcopy(g)
        start = timer()
        method(gCopy)
        end = timer()
        sum += end - start
    return sum / TIMER_TEST_COUNT

In [40]:
def moveInGraph(g: Graph, dest: int) -> Graph | None:
    if dest not in g.adj[g.current]:
        return None
    newGraph = deepcopy(g)
    newGraph.current = dest
    newGraph.cost += 1
    newGraph.parent = g
    if dest in newGraph.difficulties:
        newGraph.difficulties[dest] += 1
        newGraph.remainingTime = newGraph.difficulties[dest]
    for _, vertices in newGraph.requirements.items():
        if dest in vertices:
            vertices.remove(dest)
    if dest in newGraph.requirements and len(newGraph.requirements[dest]) == 0:
        newGraph.requirements.pop(dest)
    return newGraph


def isGoal(g: Graph) -> bool:
    return len(g.requirements) == 0


def getPath(g: Graph | None) -> list[int]:
    path = []
    while g is not None:
        path.append(g.current + 1)
        g = g.parent
    path.reverse()
    return path

In [41]:
def bfs(g: Graph) -> tuple[list[int] | None, int | None, int]:
    if isGoal(g):
        return getPath(g), g.cost,1
    discovered = set()
    queue = deque([g])
    discovered.add(str(g))
    while queue:
        g = queue.popleft()
        if g.remainingTime > 0:
            g.remainingTime -= 1
            g.cost += 1
            queue.append(g)
            continue
        for dest in g.adj[g.current]:
            newState = moveInGraph(g, dest)
            if newState is None:
                continue
            if isGoal(newState):
                return getPath(newState), newState.cost, len(discovered)
            if str(newState) not in discovered:
                queue.append(newState)
                discovered.add(str(newState))
    return None, None, len(discovered)

<div dir='rtl'>
می‌دانیم که برای عبور از هر یال به 1 ثانیه زمان نیاز داریم. از طرفی اگر در راس صعب العبوری باشیم که
n
ثانیه از زمان خروج از راس باقی مانده باشد، به اندازه
n
واحد عمق این 
state
را افزایش می‌دهیم. برای مثال فرض کنید که در حال حاضر در
stateای
باشیم که راس فعلی سید یک راس صعب العبور باشد و زمان باقی‌مانده از زمان خروج از راس برابر با 5 باشد، این
state
در الگوریتم 
BFS
تا 5 عمق پایین‌تر تکرار می‌شود و از عمق ششم می‌توانیم 
stateهایی
که از حرکت سید به راس‌های همسایه ساخته می‌شوند را در نظر بگیریم. این مورد باعث می‌شود از طریق رئوس صعب العبور راه اشتباهی با هزینه بیشتر طی نکنیم. پس در نهایت می‌توان گفت اندازه هر یال برابر با 1 واحد است و الگوریتم
BFS
پاسخ بهینه را به ما می‌دهد.
</div>

In [57]:
for i, g in enumerate(inputs):
    gCopy = deepcopy(g)
    print(f'BFS: Input {i + 1}')
    path, cost, stateCount = bfs(gCopy)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
        print('Total Visited States:', stateCount)
        print(f'Time: {searchTime(g, bfs):.4f} seconds')
    else:
        print('No solution')
    print()

BFS: Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 78
Time: 0.0069 seconds

BFS: Input 2
Path: 9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Total Visited States: 115
Time: 0.0098 seconds

BFS: Input 3
Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Total Visited States: 1769
Time: 0.1697 seconds

BFS: Input 4
Path: 28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 2 -> 5 -> 7 -> 29 -> 22 -> 28
Cost: 12
Total Visited States: 7536
Time: 1.5185 seconds

BFS: Input 5
Path: 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Cost: 21
Total Visited States: 10630
Time: 1.2077 seconds



<div dir='rtl'>
همانطور که مشاهده می‌شود، زمان پاسخ دادن تست‌ها بسیار کمتر از محدودیت زمانی ذکر شده در صورت سوال است.
</div>

In [43]:
def dfs(g: Graph, depth: int, discovered: set, visited: set) -> Graph | None:
    if isGoal(g):
        return g
    if depth <= 0:
        return None
    remainingTime = g.remainingTime
    if remainingTime > 0:
        g.remainingTime = 0
        g.cost += remainingTime
        if depth <= remainingTime + 1:
            return None
    children = set()
    for dest in g.adj[g.current]:
        newState = moveInGraph(g, dest)
        if newState is None:
            continue
        if str(newState) in visited:
            continue
        children.add(((newState, str(newState))))
        discovered.add(str(newState))
        visited.add(str(newState))
    for child, _ in children:
        ans = dfs(child, depth - 1 - remainingTime, discovered, visited)
        if ans is not None:
            return ans
    
    for _, key in children:
        visited.remove(str(key))
    return None


def ids(g: Graph) -> tuple[list[int] | None, int | None, int]:
    depth = 0
    while True:
        discovered = set()
        discovered.add(str(g))
        visited = set()
        visited.add(str(g))
        ans = dfs(g, depth, discovered, visited)
        if ans is not None:
            return getPath(ans), ans.cost, len(discovered)
        # print(f'IDS: Depth {depth} - Visited States: {len(discovered)}')
        depth += 1

<div dir='rtl'>
الگوریتم
DFS
در حالت عادی پاسخ بهینه را به ما نمی‌دهد. زیرا ممکن است در شاخه اول پاسخی بلندتر از شاخه دوم یافت شود. اما زمانی که از 
IDS
استفاده می‌کنیم، این الگوریتم پاسخ بهینه را پیدا می‌کند. با توجه به اینکه عمق را 1 واحد 1 واحد افزایش می‌دهیم، اگر جواب کوتاه‌تری وجود داشت، در همان عمق به جواب می‌رسیدیم.<br/>
این الگوریتم تعداد 
stateهای
بسیار زیادی (افزایش با مرتبه نمایی) تولید می‌کند که درصد ناچیزی از آن‌ها غیر تکراری هستند. <br/>
اگر خط کامنت شده در تابع را از کامنت در بیاوریم، می‌توانیم تعداد 
stateهای
غیر تکراری را در هر عمق مشاهده کنیم. همچنین اگر 
discovered
را از
set
به 
list
تغییر دهیم، می‌توانیم تعداد تمام
stateهای
هر عمق را مشاهده کنیم. این اعداد برای تست شماره 4 به صورت زیر است:
<ul>
    <li>عمق 6: حدود 90 هزار state که حدود 1000تای آن‌ها غیر تکراری هستند</li>
    <li>عمق 7: حدود 660 هزار state که حدود 2000تای آن‌ها غیر تکراری هستند</li>
    <li>عمق 8: حدود 4.8 میلیون state که حدود 3000تای آن‌ها غیر تکراری هستند</li>
</ul>
لازم به ذکر است که پاسخ نهایی در عمق 12 قرار دارد که اگر تعداد
stateها
به همین شیوه به صورت نمایی ادامه پیدا کند، تعداد
stateها
در عمق 12 به بیش از 5 میلیارد می‌رسد. به همین دلیل زمان اجرایی این الگوریتم برای تست‌های 4 و 5 بسیار زیاد است.
</div>

In [58]:
for i, g in enumerate(inputs[:3]): # remove [:3] to run all inputs (takes a long time =] )
    gCopy = deepcopy(g)
    print(f'IDS: Input {i + 1}')
    path, cost, stateCount = ids(gCopy)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
        print('Total Visited States:', stateCount)
        print(f'Time: {searchTime(g, ids):.4f} seconds')
    else:
        print('No solution')
    print()

IDS: Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 76
Time: 0.0135 seconds

IDS: Input 2
Path: 9 -> 10 -> 2 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Total Visited States: 88
Time: 0.0541 seconds

IDS: Input 3
Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Total Visited States: 1521
Time: 1.5436 seconds



<div dir='rtl'>
با وجود اینکه در صورت سوال محدودیت زمانی برای این الگوریتم ذکر نشده است، زمان پاسخگویی به تست‌ها بسیار معقول است.
همچنین برای تست‌های 4 و 5 با توجه به طولانی بودن زمان اجرا، تصمیم به اجرای آن‌ها در یک سرور شخصی (با سی پی یو تک هسته‌ای) گرفتم که نتیجه آن در بخش زیر قابل مشاهده است.<br/>
لازم به ذکر است که چون همسایه‌های راس 
explore
شده در یک ست قرار می‌گیرند (ترتیب ثابتی وجود ندارد) و سپس تابع 
DFS
بر روی آن‌ها صدا زده می‌شود، مسیر یافت شده و تعداد 
stateهای
پردازش شده لزوما با هر بار اجرا ثابت نخواهند ماند و ممکن است تغییر یابند.<br/>
همچنین، تعداد
stateهای
نمایش داده شده فقط مربوط به
stateهای
غیر تکراری در <b>عمق آخر</b> است.
</div>

![IDS](./assets/IDS%20Results.png)

In [50]:
def heuristic(g: Graph) -> int:
    remainingCount = set()
    for r, vertices in g.requirements.items():
        remainingCount.add(r)
        remainingCount.update(vertices)
    return len(remainingCount)

def aStar(g: Graph, alpha: float = 1) -> tuple[list[int] | None, int | None, int]:
    q = []
    heapq.heappush(q, (0, g))
    discovered = set()
    discovered.add(str(g))
    while q:
        _, g = heapq.heappop(q)
        if isGoal(g):
            return getPath(g), g.cost, len(discovered)
        if g.remainingTime > 0:
            g.remainingTime -= 1
            g.cost += 1
            heapq.heappush(q, (g.cost + alpha * heuristic(g), g))
            continue
        for dest in g.adj[g.current]:
            newState = moveInGraph(g, dest)
            if newState is None:
                continue
            if str(newState) not in discovered:
                heapq.heappush(q, (newState.cost + alpha * heuristic(newState), newState))
                discovered.add(str(newState))
    return None, None, len(discovered)

<div dir='rtl'>
برای هیوریستیک مورد زیر را در نظر می‌گیریم:<br/>
تعداد مریدها و دستور غذاهای باقی مانده (این رئوس را به صورت یکتا در نظر می‌گیریم زیرا ممکن است در یک راس چند مرید و چند دستور غذا باقی مانده باشد)<br/>
برای اثبات 
consistent
بودن این هیوریستیک به شکل زیر عمل می‌کنیم:<br/>
دو
state
را با نام‌های
A و C
در نظر بگیرید به طوری که 
A
یکی از جدهای
C
است. 
حال فرض کنید که تعداد رئوس (مریدها و دستور غذاها) باقی مانده در 
A
برابر با 
n
است و تعداد رئوس باقی مانده در
C
برابر با
m
باشد. در این صورت هنگام رسیدن از 
A
به
C
باید حداقل از
n-m
راس بگذریم. با توجه به اینکه هزینه هر یال برابر با 1 واحد است، می‌توان گفت
cost(A to C) &GreaterEqual; n-m
از طرفی می‌دانیم که هیوریستیک ذکر شده برای هر 
state
برابر با تعداد رئوس باقی مانده است به طوری که می‌توان گفت
h(A) = n
و
h(C) = m
و در نتیجه به 
cost(A to C) &GreaterEqual; h(A) - h(C)
می‌رسیم. با توجه به این مورد، اثبات می‌شود که این هیوریستیک 
consistent
است.
<br/>
لازم به ذکر است که می‌توانستیم از 
MST
به عنوان هیوریستیک استفاده کنیم به این صورت که برای هر 
state
یک درخت کمینه تشکیل می‌دادیم که نیازی نیست برای کل رئوس گراف پوشا باید بلکه لازم است برای رئوس باقی مانده (مریدها و دستور غذاها) پوشا باشد. در این الگوریتم هر زمانی که نیاز باشد می‌توانیم از رئوس دیگر استفاده کنیم. این هیوریستیک گرچه هیوریستیک قوی‌تری است، اما زمان زیادی برای محاسبه نیاز دارد که در نهایت مرتبه زمانی الگوریتم را افزایش می‌دهد.
</div>

<div dir='rtl'>
الگوریتم
*A
جواب بهینه را محاسبه می‌کند. البته لازم به ذکر است که برای این کار لازم است 2 مورد رعایت شود:
<ol>
    <li>هیوریستیک مورد استفاده باید consistent باشد.</li>
    <li>تست goal باید زمان پاپ شدن state از frontier انجام شود زیرا اگر این تست را زمان اضافه کردن state به frontier انجام دهیم، لزوما پاسخ ما بهینه نخواهد بود.</li>
</ol>
با توجه به اینکه هر 2 مورد ذکر شده در الگوریتم رعایت شده است، می‌توان گفت الگوریتم
*A
پاسخ بهینه را به دست می‌آورد.
</div>

In [49]:
for i, g in enumerate(inputs):
    gCopy = deepcopy(g)
    print(f'A*: Input {i + 1}')
    path, cost, stateCount = aStar(gCopy)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
        print('Total Visited States:', stateCount)
        print(f'Time: {searchTime(g, lambda g: aStar(g, 1)):.4f} seconds')
    else:
        print('No solution')
    print()

A*: Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 78
Time: 0.0028 seconds

A*: Input 2
Path: 9 -> 10 -> 2 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Total Visited States: 88
Time: 0.0089 seconds

A*: Input 3
Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Total Visited States: 711
Time: 0.0576 seconds

A*: Input 4
Path: 28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 27 -> 5 -> 7 -> 29 -> 22 -> 28
Cost: 12
Total Visited States: 2670
Time: 0.4009 seconds

A*: Input 5
Path: 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Cost: 21
Total Visited States: 7894
Time: 1.1771 seconds



<div dir='rtl'>
همانطور که مشاهده می‌شود، الگوریتم
*A
تست‌های ورودی را در زمان معقول و بسیار کمتر از محدودیت زمانی صورت سوال، پاسخ داده است.
</div>

<div dir='rtl'>
در کل می‌توان گفت بین تمامی الگوریتم‌هایی که تا به اینجا استفاده شدند، همگی پاسخ بهینه را بدست می‌آورند اما الگوریتم
*A
در کمترین زمان ممکن پاسخ را به دست می‌آورد.<br/>
تفاوت بعدی بین الگوریتم‌های استفاده شده، حافظه مصرفی الگوریتم‌ها است. الگوریتم‌های
BFS و *A
به صورت 
exponential
حافظه مصرف می‌کنند. در صورتی که حافظه مصرفی الگوریتم
IDS
به صورت
خطی یا به عبارت دیگر
polynomial
است. در نتیجه در این الگوریتم مشکل حافظه نخواهیم داشت در صورتی که در دو الگوریتم قبلی ممکن است در تست‌های بزرگ، مشکل حافظه رخ دهد.
</div>

In [52]:
for i, g in enumerate(inputs):
    gCopy = deepcopy(g)
    print(f'A* (alpha = 1.2): Input {i + 1}')
    path, cost, stateCount = aStar(gCopy, 1.2)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
        print('Total Visited States:', stateCount)
        print(f'Time: {searchTime(g, lambda g: aStar(g, 2)):.4f} seconds')
    else:
        print('No solution')
    print()

A* (alpha = 1.2): Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 52
Time: 0.0012 seconds

A* (alpha = 1.2): Input 2
Path: 9 -> 10 -> 2 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Total Visited States: 55
Time: 0.0037 seconds

A* (alpha = 1.2): Input 3
Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Total Visited States: 425
Time: 0.0089 seconds

A* (alpha = 1.2): Input 4
Path: 28 -> 23 -> 9 -> 24 -> 11 -> 3 -> 13 -> 23 -> 5 -> 7 -> 29 -> 22 -> 28
Cost: 12
Total Visited States: 436
Time: 0.0055 seconds

A* (alpha = 1.2): Input 5
Path: 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Cost: 21
Total Visited States: 6565
Time: 0.3648 seconds



<div dir='rtl'>
همانطور که مشاهده می‌شود، در الگوریتم 
Weighted *A
تعداد 
stateهای
مشاهده شده و زمان اجرا به مراتب کمتر از الگوریتم
*A
است. این اتفاق به دلیل این است که در این الگوریتم هیوریستیک مورد استفاده در عددی ضرب می‌شود زیرا می‌دانیم اگر یک هیوریستیک
consistent
باشد، همواره کمتر از هزینه واقعی خواهد بود و در نتیجه با ضرب عددی در آن تلاش می‌کنیم این مقدار را به هزینه واقعی نزدیک‌تر کنیم.<br/>
این الگوریتم در این حالت خاص پاسخ بهینه را به ما داده است ولی لزوما همیشه بهینه نخواهد بود.
</div>

In [51]:
for i, g in enumerate(inputs):
    gCopy = deepcopy(g)
    print(f'A* (alpha = 1.4): Input {i + 1}')
    path, cost, stateCount = aStar(gCopy, 1.4)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
        print('Total Visited States:', stateCount)
        print(f'Time: {searchTime(g, lambda g: aStar(g, 2)):.4f} seconds')
    else:
        print('No solution')
    print()

A* (alpha = 1.4): Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 52
Time: 0.0021 seconds

A* (alpha = 1.4): Input 2
Path: 9 -> 10 -> 2 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Total Visited States: 46
Time: 0.0015 seconds

A* (alpha = 1.4): Input 3
Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Total Visited States: 254
Time: 0.0084 seconds

A* (alpha = 1.4): Input 4
Path: 28 -> 19 -> 3 -> 11 -> 24 -> 9 -> 2 -> 5 -> 7 -> 29 -> 20 -> 13 -> 23 -> 28
Cost: 13
Total Visited States: 213
Time: 0.0052 seconds

A* (alpha = 1.4): Input 5
Path: 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Cost: 21
Total Visited States: 5752
Time: 0.4214 seconds



<div dir='rtl'>
همانطور که مشاهده می‌شود، به ازای
&alpha; = 1.4،
الگوریتم
Weighted *A
پاسخ بهینه را به ما نداده است. در تست شماره 4، عمق پاسخ بهینه 12 است در صورتی که پاسخ محاسبه شده در این بخش در عمق 13 قرار دارد.
</div>

<div dir='rtl'>
در نهایت می‌توان گفت الگوریتم
Weighted *A
با انتخاب ضریب مناسب، در کمترین زمان ممکن پاسخ را محاسبه می‌کند ولی این پاسخ لزوما بهینه نیست.
</div>

<div dir='rtl'>
جداول مربوط به تست کیس‌ها در بخش زیر آورده شده‌اند. لازم به ذکر است که تعداد 
stateهای
دیده شده در هر تست کیس، تعداد
stateهای
<b>غیر تکراری</b>
است.
</div>

<div dir='rtl'>
<h2>تست شماره 1 (تست اول در تست کیس‌های اصلی و ساده)</h2>
</div>

|                            | Minimum Cost | Number of Visited States | Execution Time |
|         :----:             |    :----:    |          :----:          |     :----:     |
|          BFS               |      8       |            78            |     0.0069s    |
|          IDS               |      8       |            76            |     0.0135s    |
|           A*               |      8       |            78            |     0.0028s    |
| Weighted A* $(\alpha=1.2)$ |      8       |            52            |     0.0012s    |
| Weighted A* $(\alpha=1.4)$ |      8       |            52            |     0.0021s    |

<div dir='rtl'>
<h2>تست شماره 2 (تست دوم در تست کیس‌های ساده)</h2>
</div>

|                            | Minimum Cost | Number of Visited States | Execution Time |
|         :----:             |    :----:    |          :----:          |     :----:     |
|          BFS               |      8       |           115            |     0.0098s    |
|          IDS               |      8       |            88            |     0.0541s    |
|           A*               |      8       |            88            |     0.0089s    |
| Weighted A* $(\alpha=1.2)$ |      8       |            55            |     0.0037s    |
| Weighted A* $(\alpha=1.4)$ |      8       |            46            |     0.0015s    |

<div dir='rtl'>
<h2>تست شماره 3 (تست سوم در تست کیس‌های ساده)</h2>
</div>

|                            | Minimum Cost | Number of Visited States | Execution Time |
|         :----:             |    :----:    |          :----:          |     :----:     |
|          BFS               |      13      |           1769           |     0.1697s    |
|          IDS               |      13      |           1521           |     1.5436s    |
|           A*               |      13      |           711            |     0.0576s    |
| Weighted A* $(\alpha=1.2)$ |      13      |           425            |     0.0089s    |
| Weighted A* $(\alpha=1.4)$ |      13      |           254            |     0.0084s    |

<div dir='rtl'>
<h2>تست شماره 4 (تست دوم در تست کیس‌های اصلی)</h2>
</div>

|                            | Minimum Cost | Number of Visited States | Execution Time |
|         :----:             |    :----:    |          :----:          |     :----:     |
|          BFS               |      12      |           7536           |     1.5185s    |
|          IDS               |      12      |           4996           |    1h29m14s    |
|           A*               |      12      |           2670           |     0.4009s    |
| Weighted A* $(\alpha=1.2)$ |      12      |           436            |     0.0055s    |
| Weighted A* $(\alpha=1.4)$ |      13      |           213            |     0.0052s    |

<div dir='rtl'>
<h2>تست شماره 5 (تست سوم در تست کیس‌های اصلی)</h2>
</div>

|                            | Minimum Cost | Number of Visited States | Execution Time |
|         :----:             |    :----:    |          :----:          |     :----:     |
|          BFS               |      21      |           10630          |     1.2077s    |
|          IDS               |              |                          |                |
|           A*               |      21      |           7894           |     1.1771s    |
| Weighted A* $(\alpha=1.2)$ |      21      |           6565           |     0.3648s    |
| Weighted A* $(\alpha=1.4)$ |      21      |           5752           |     0.4214s    |